In [33]:
# %pip install --upgrade openai

In [12]:
from dotenv import load_dotenv
load_dotenv()

import os
from openai import OpenAI
from gpt import (
    create_file_search_assistant,
    upload_file_to_vectore_store,
    assistant_use_vector_store,
    create_thread,
)

In [13]:
MODEL = "gpt-4o"
FILE = "giec.pdf"
client = OpenAI(
    api_key=os.environ.get("API_KEY")
)

In [14]:
assistant = create_file_search_assistant(
    client=client,
    model=MODEL,
    assistant_name="Environment Analyst Assistant",
    instructions="You are an environment expert analyst. Use you knowledge base to extract and summarize information about audited environment documents",
)

create_file_search_assistant took 0.46157050132751465 seconds


In [15]:
vector_store = upload_file_to_vectore_store(
    client=client,
    vector_store_name="Environment Reports",
    file_paths=[FILE],
)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)
upload_file_to_vectore_store took 7.98323917388916 seconds


In [16]:
assistant = assistant_use_vector_store(
    client=client,
    assistant=assistant,
    vector_store=vector_store,
)

assistant_use_vector_store took 0.36090874671936035 seconds


In [17]:
thread = create_thread(
    client=client,
    message_content="Make a short summary with the key points of the document",
)

None
create_thread took 0.22078442573547363 seconds


In [18]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("="*50)
print("\n".join(citations))

The document is a special report by the Intergovernmental Panel on Climate Change (IPCC) concerning the impacts of global warming of 1.5°C above pre-industrial levels and related global greenhouse gas emission pathways. Here are the key points:

1. **Scope and Purpose**:
   - The report was organized in response to an invitation by the Conference of the Parties (COP) to the United Nations Framework Convention on Climate Change (UNFCCC) as part of the Paris Agreement. It assesses the impacts of 1.5°C of global warming and the associated emission trajectories necessary to achieve this target[0].

2. **Report Structure**:
   - The report consists of a Summary for Policymakers, Technical Summary, five chapters, and supplementary online materials. The chapters cover the evaluation methods, current climate system status, impacts and risks of 1.5°C warming, global and socio-economic transitions required, and the links with sustainable development and poverty eradication[0][2].

3. **Emission 